In [1]:
import pandas as pd
import requests
import numpy as np
from nba_py import game, team
import json

In [2]:
# get individual game play by play
# result = game.PlayByPlay('0021600001')
# # print(result.json['resultSets'][0])
# headers = (result.json['resultSets'][0]['headers'])
# values = (result.json['resultSets'][0]['rowSet'])
# df = pd.DataFrame(values, columns=headers)
# df.shape

In [3]:
# gat game ids for a team
def getGameIDs(teamID):
    result = team.TeamGameLogs(teamID)
    values = (result.json['resultSets'][0]['rowSet'])
    gameList =  [v[1] for v in values]
    locs = [v[3] for v in values]
    locsList = []
    for l in locs:
        if l.find('@') > 0:
            locsList.append('VISITOR')
        else:
            locsList.append('HOME')
    return gameList, locsList
    
gameIDS, locList = getGameIDs(1610612738)
locDict = dict(zip(gameIDS, locList))
locDict

{'0021700001': 'VISITOR',
 '0021700007': 'HOME',
 '0021700019': 'VISITOR',
 '0021700051': 'HOME',
 '0021700067': 'VISITOR',
 '0021700078': 'VISITOR',
 '0021700092': 'HOME',
 '0021700109': 'HOME',
 '0021700129': 'VISITOR',
 '0021700138': 'VISITOR',
 '0021700146': 'VISITOR',
 '0021700161': 'HOME',
 '0021700170': 'HOME',
 '0021700188': 'HOME',
 '0021700201': 'VISITOR',
 '0021700215': 'HOME',
 '0021700230': 'VISITOR',
 '0021700249': 'VISITOR',
 '0021700257': 'VISITOR',
 '0021700269': 'HOME',
 '0021700281': 'VISITOR',
 '0021700292': 'HOME',
 '0021700314': 'HOME',
 '0021700326': 'HOME',
 '0021700343': 'HOME',
 '0021700356': 'HOME',
 '0021700374': 'VISITOR',
 '0021700386': 'VISITOR',
 '0021700391': 'VISITOR',
 '0021700407': 'HOME',
 '0021700423': 'HOME',
 '0021700436': 'VISITOR',
 '0021700442': 'VISITOR',
 '0021700456': 'HOME',
 '0021700468': 'VISITOR',
 '0021700485': 'HOME',
 '0021700495': 'HOME',
 '0021700506': 'VISITOR',
 '0021700517': 'HOME',
 '0021700539': 'HOME',
 '0021700559': 'HOME',


In [4]:
def getPBPData(gameList, locList):
    headers = []
    data = []
    for idx, g in enumerate(gameList):
    # get individual game play by play
        result = game.PlayByPlay(g)
        if (len(headers) == 0):
            headers = (result.json['resultSets'][0]['headers'])
        values = (result.json['resultSets'][0]['rowSet'])
        data.extend(values)
    return pd.DataFrame(data, columns=headers)

df = getPBPData(gameIDS, locList)


In [9]:
df.drop(['EVENTMSGTYPE','WCTIMESTRING','EVENTMSGACTIONTYPE'],axis=1, inplace=True)
df['LOCATION'] = df.GAME_ID
df.replace({"LOCATION": locDict}, inplace=True)


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,LOCATION
37352,0021700001,670,2,1,4,10:32 PM,0:02,None,None,MISS Brown 25' 3PT Jump Shot,None,None,VISITOR
37353,0021700001,671,4,0,4,10:32 PM,0:02,None,None,Irving REBOUND (Off:2 Def:2),None,None,VISITOR
37354,0021700001,672,2,1,4,10:32 PM,0:00,None,None,MISS Irving 26' 3PT Jump Shot,None,None,VISITOR
37355,0021700001,673,4,0,4,10:32 PM,0:00,None,None,Celtics Rebound,None,None,VISITOR
37356,0021700001,674,13,0,4,10:32 PM,0:00,None,None,None,99 - 102,3,VISITOR


In [10]:
df['ISHOMEMISS3PT']= df.HOMEDESCRIPTION.str.contains('^MISS.+3PT')
df['HOMEMISS2PTDIST'] = df.HOMEDESCRIPTION.str.extract('^MISS.+ (\d{1,2})\' (?!3PT)')
df['ISVISITORMISS3PT']= df.VISITORDESCRIPTION.str.contains('^MISS.+3PT')
df['VISITORMISS2PTDIST'] = df.VISITORDESCRIPTION.str.extract('^MISS.+ (\d{1,2})\' (?!3PT)')

df.loc[df.ISHOMEMISS3PT == True, 'HOMEMISSALL'] = '3PT'
df.loc[df.HOMEMISS2PTDIST.notnull(), 'HOMEMISSALL'] = df.HOMEMISS2PTDIST[df.HOMEMISS2PTDIST.notnull()]
df.loc[df.ISVISITORMISS3PT == True, 'VISITORMISSALL'] = '3PT'
df.loc[df.VISITORMISS2PTDIST.notnull(), 'VISITORMISSALL'] = df.VISITORMISS2PTDIST[df.VISITORMISS2PTDIST.notnull()]

# shift misses down one to get event after shot
df['HOMEMISSSHIFT'] = df.HOMEMISSALL.shift()
# filter event description on None = Def, PLayer REBOUND (count) = Off, Team Rebound = None
df['HOMEMISSRBDTYPE'] =  np.where(df['HOMEMISSSHIFT'].notnull(), # filter on misses
                                (np.where(df['HOMEDESCRIPTION'].notnull(), # filter on non null event description = DRBD
                                np.where(df['HOMEDESCRIPTION'].str.contains(("\(.+\)")),'OFF', None),'DEF')), # either team rebound or str contains (.+) = ORBD
                                  None)

# shift misses down one to get event after shot
df['VISITORMISSSHIFT'] = df.VISITORMISSALL.shift()
# filter event description on None = Def, PLayer REBOUND (count) = Off, Team Rebound = None
df['VISITORMISSRBDTYPE'] =  np.where(df['VISITORMISSSHIFT'].notnull(), # filter on misses
                                (np.where(df['VISITORDESCRIPTION'].notnull(), # filter on non null event description = DRBD
                                np.where(df['VISITORDESCRIPTION'].str.contains(("\(.+\)")),'OFF', None),'DEF')), # either team rebound or str contains (.+) = ORBD
                                  None)



# df.tail(50)

C:\Users\swlaforest\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
C:\Users\swlaforest\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,ISHOMEMISS3PT,HOMEMISS2PTDIST,ISVISITORMISS3PT,VISITORMISS2PTDIST,HOMEMISSALL,VISITORMISSALL,HOMEMISSSHIFT,HOMEMISSRBDTYPE,VISITORMISSSHIFT,VISITORMISSRBDTYPE
37307,0021700001,606,5,1,4,10:17 PM,5:11,Shumpert Bad Pass Turnover (P1.T16),None,Rozier STEAL (4 STL),...,False,NaN,False,NaN,NaN,NaN,NaN,None,NaN,None
37308,0021700001,642,6,1,4,10:17 PM,5:11,Shumpert P.FOUL (P2.PN) (M.McCutchen),None,None,...,False,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None
37309,0021700001,649,3,12,4,10:17 PM,5:11,None,None,Rozier Free Throw 2 of 2 (8 PTS),...,None,NaN,False,NaN,NaN,NaN,NaN,None,NaN,None
37310,0021700001,648,3,11,4,10:17 PM,5:11,None,None,Rozier Free Throw 1 of 2 (9 PTS),...,None,NaN,False,NaN,NaN,NaN,NaN,None,NaN,None
37311,0021700001,610,8,0,4,10:17 PM,5:11,SUB: Rose FOR Shumpert,None,None,...,False,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None
37312,0021700001,611,8,0,4,10:17 PM,5:11,None,None,SUB: Horford FOR Baynes,...,None,NaN,False,NaN,NaN,NaN,NaN,None,NaN,None
37313,0021700001,612,5,40,4,10:18 PM,4:58,Love Out of Bounds Lost Ball Turnover (P2.T17),None,None,...,False,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None
37314,0021700001,615,8,0,4,10:18 PM,4:58,None,None,SUB: Smart FOR Rozier,...,None,NaN,False,NaN,NaN,NaN,NaN,None,NaN,None
37315,0021700001,616,2,5,4,10:18 PM,4:41,Wade BLOCK (2 BLK),None,MISS Horford 5' Layup,...,False,NaN,False,5,NaN,5,NaN,None,NaN,None
37316,0021700001,617,4,0,4,10:18 PM,4:41,Love REBOUND (Off:2 Def:8),None,None,...,False,NaN,None,NaN,NaN,NaN,NaN,None,5,DEF


In [15]:
df.head()

,GAME_ID,EVENTNUM,PERIOD,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,LOCATION,ISHOMEMISS3PT,HOMEMISS2PTDIST,ISVISITORMISS3PT,VISITORMISS2PTDIST,HOMEMISSALL,VISITORMISSALL,HOMEMISSSHIFT,HOMEMISSRBDTYPE,VISITORMISSSHIFT,VISITORMISSRBDTYPE
0,0021701219,2,1,12:00,None,None,None,None,None,HOME,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None
1,0021701219,4,1,12:00,Jump Ball Baynes vs. Allen: Tip to Larkin,None,None,None,None,HOME,False,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None
2,0021701219,7,1,11:47,MISS Baynes 16' Jump Shot,None,None,None,None,HOME,False,16,None,NaN,16,NaN,NaN,None,NaN,None
3,0021701219,8,1,11:42,None,None,Allen REBOUND (Off:0 Def:1),None,None,HOME,None,NaN,False,NaN,NaN,NaN,16,DEF,NaN,None
4,0021701219,9,1,11:23,None,None,MISS Russell 15' Pullup Jump Shot,None,None,HOME,None,NaN,False,15,NaN,15,NaN,None,NaN,None


0.27405559515324307

In [13]:
df[df.HOMEMISSRBDTYPE == 'DEF'].shape

(2806, 20)

In [14]:
df.GAME_ID.nunique()

82